In [1]:
import json

def print_json(filename: str):
    with open(filename, 'r') as openfile:
        json_object = json.load(openfile)

    json_object = json.dumps(json_object, indent=2)
    print(json_object)
    
def print_txt(filename: str):
    with open(filename, 'r') as openfile:
        print(openfile.read())

## Introduction example

In [2]:
from rtb_argparse.config import ArgparseConfig, ParserJson

parser =  ArgparseConfig(fromfile_prefix_chars="@", file_parser=ParserJson().parser)
parser.add_argument('arg1', type=str)
parser.add_argument("--foo", type=str)
parser.add_argument("--name", type=str)

parser.print_help()

usage: ipykernel_launcher.py [-h] [--foo FOO] [--name NAME] arg1

positional arguments:
  arg1

options:
  -h, --help   show this help message and exit
  --foo FOO
  --name NAME


### - Simple example

In [3]:
filename = "data/introduction_simple_example.json"
print_json(filename)

[
  "text",
  {
    "--foo": "foo",
    "--name": "myname"
  }
]


In [4]:
opt = parser.parse_args([f"@{filename}"])
print(opt.arg1, opt.foo, opt.name)

text foo myname


### - Complex example

In [5]:
filename = "data/introduction_complex_example.json"
print_json(filename)

[
  "text",
  {
    "@default": {
      "--foo": "foo",
      "--name": "myname"
    },
    "@conf1": {
      "--foo": "oof",
      "--name": "conf1"
    }
  }
]


In [6]:
opt = parser.parse_args([f"@{filename}"])
print(opt.arg1, opt.foo, opt.name)

text foo myname


In [7]:
opt = parser.parse_args([f"@{filename}@conf1"])
print(opt.arg1, opt.foo, opt.name)

text oof conf1


## Config

### - default_file_parser

In [8]:
filename = "data/default_file_parser.txt"
print_txt(filename)

text --foo foo
--name myname


In [9]:
from rtb_argparse.config import ArgparseConfig

# default_file_parser is the default function of file_parser so there is no need to specify it.
parser =  ArgparseConfig(fromfile_prefix_chars="@") 
parser.add_argument('arg1', type=str)
parser.add_argument("--foo", type=str)
parser.add_argument("--name", type=str)

opt = parser.parse_args([f"@{filename}"])
print(opt.arg1, opt.foo, opt.name)

text foo myname


### - config_parser

In [10]:
filename = "data/config_parser.conf"
print_txt(filename)

# This comment is just here to show how to comment

"text" # It's arg1

@default --foo foo
--name myname

@conf1
--foo oof
--name conf1


In [11]:
from rtb_argparse.config import ArgparseConfig, ParserConfig

parser =  ArgparseConfig(fromfile_prefix_chars="@", file_parser=ParserConfig(verbose=True).parser) 
parser.add_argument('arg1', type=str)
parser.add_argument("--foo", type=str)
parser.add_argument("--name", type=str)

opt = parser.parse_args([f"@{filename}"])
print(opt.arg1, opt.foo, opt.name)

print()

opt = parser.parse_args([f"@{filename}@conf1"])
print(opt.arg1, opt.foo, opt.name)

Reading args from 'data/config_parser.conf' with config 'default'
"text" foo myname

Reading args from 'data/config_parser.conf' with config 'conf1'
"text" oof conf1


## Formatters

`argparse.ArgumentDefaultsHelpFormatter` does not displays the default value if an argument does not have a help string, `formatters.ArgumentDefaultsHelpFormatter` does.

In [12]:
import argparse
from rtb_argparse import formatters

print("### argparse Help Formatter\n")
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--var1', default="it's a string")
parser.add_argument('--var2', default="it's another string", help="It's var2")
parser.print_help()

print("\n### rtb_argparse Help Formatter\n")
parser = argparse.ArgumentParser(formatter_class=formatters.ArgumentDefaultsHelpFormatter)
parser.add_argument('--var1', default="it's a string")
parser.add_argument('--var2', default="it's another string", help="It's var2")
parser.print_help()

### argparse Help Formatter

usage: ipykernel_launcher.py [-h] [--var1 VAR1] [--var2 VAR2]

options:
  -h, --help   show this help message and exit
  --var1 VAR1
  --var2 VAR2  It's var2 (default: it's another string)

### rtb_argparse Help Formatter

usage: ipykernel_launcher.py [-h] [--var1 VAR1] [--var2 VAR2]

options:
  -h, --help   show this help message and exit
  --var1 VAR1  (default: it's a string)
  --var2 VAR2  It's var2 (default: it's another string)


## Checkers

In [13]:
import argparse
from rtb_argparse import checkers

parser = argparse.ArgumentParser()

# Test if x >= 0
parser.add_argument('var1', type=lambda x: checkers.check_superior(int(x), 0))
# Test if x is a file and if its extension is txt
parser.add_argument('var2', type=lambda x: checkers.check_file_extension(checkers.check_is_file(x), 'ipynb'))
# Eval the x string with ast.literal_eval and test if the result is a dict
parser.add_argument('var3', type=lambda x: checkers.check_type(checkers.do_literal_eval(x), dict))

opt = parser.parse_args(["5", "readme_examples.ipynb", "{'test': 'working!'}"])

In [14]:
print(opt.var1, opt.var2, opt.var3)
print(type(opt.var1), type(opt.var2), type(opt.var3))

5 readme_examples.ipynb {'test': 'working!'}
<class 'int'> <class 'str'> <class 'dict'>
